# Selective Inference via outcome Randomisation and Mirror Statistics

In [12]:
using Pkg
Pkg.status()

using GLM
using GLMNet
using Distributions
using Random
using StatsPlots
using Plots
using DataFrames
using CSV

Project selective_inference_project v0.1.0
Status `~/Documents/UiO_Postdoc/Code/git_repositories/SelectiveInference/Project.toml`
  [336ed68f] CSV v0.10.11
⌃ [31c24e10] Distributions v0.25.100
⌃ [38e38edf] GLM v1.8.3
  [8d5ece8b] GLMNet v0.7.2
⌃ [6f49c342] RCall v0.13.17
  [f3b207a7] StatsPlots v0.15.6
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [13]:
include("../utilities/data_generation.jl")
include("../utilities/randomisation_ds.jl")
include("../utilities/mirror_statistic.jl")
include("../utilities/classification_metrics.jl");

In [14]:
include("../wrapper_pipeline_inference.jl");

In [4]:
function print_metrics(;scenario, results)
    println("-------------------- $scenario --------------------")
    
    for (metric, value) in zip(keys(results.class_metrics), results.class_metrics)
        println("$metric ==> $value")
    end
end

print_metrics (generic function with 1 method)

In [5]:
function double_boxplot(;df, group_var, var_one, var_two, title_plot)
    max_y = maximum(
        (maximum(df[!, var_one]),
        maximum(df[!, var_two]))
    )
    min_y = minimum(
        (minimum(df[!, var_one]),
        minimum(df[!, var_two]))
    )
    l = @layout [grid(1, 2)]
    p1 = @df df boxplot(string.(cols(group_var)), cols(var_one), group=cols(group_var), label=false)
    ylims!((min_y, max_y))
    xlabel!(string(group_var))
    title!(string(var_one))
    p2 = @df df boxplot(string.(cols(group_var)), cols(var_two), group=cols(group_var), label=false)
    ylims!((min_y, max_y))
    xlabel!(string(group_var))
    title!(string(var_two))
    all_p = plot(p1, p2, layout = l)
    all_p[:plot_title] = title_plot
    plot(all_p)
end;

## Simulation on uncorrelated covariates

### Low-dimensional case
30% of coefficients are 0

In [15]:
n = 100
p = 20
prop_zero_coef = 0.3
beta_intercept = 1.
sigma2 = 1.
correlation_coefficients = []
scenario = "Low-Dimensional, No Correlation, Random Cov"

Random.seed!(1345)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=false,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    beta_intercept=beta_intercept,
    gamma_randomisation=1.,
    estimate_sigma2=true,
    fdr_level=0.1
);

In [16]:
print_metrics(scenario=scenario, results=results)

-------------------- Low-Dimensional, No Correlation, Random Cov --------------------
FDR_rand_plus_MS ==> 0.125
FDR_rand_only ==> 0.0
FDR_MS_only ==> 0.0
FDR_MDS_only ==> 0.06666666666666667
TPR_rand_plus_MS ==> 1.0
TPR_rand_only ==> 1.0
TPR_MS_only ==> 1.0
TPR_MDS_only ==> 1.0
POWER_rand_plus_MS ==> 1.0
POWER_rand_only ==> 1.0
POWER_MS_only ==> 1.0
POWER_MDS_only ==> 1.0


### High-dimensional case
70% of coefficients are 0

In [17]:
n = 500
p = 500
prop_zero_coef=0.9
beta_intercept=1.
sigma2=1.
correlation_coefficients=[]
block_covariance=false
scenario = "SAME AS PAPER - High-Dimensional, No Correlation, Random Cov"

"SAME AS PAPER - High-Dimensional, No Correlation, Random Cov"

In [18]:
Random.seed!(13)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    beta_intercept=beta_intercept,
    gamma_randomisation=1.,
    estimate_sigma2=false,
    fdr_level=0.1
);

print_metrics(scenario=scenario, results=results)

-------------------- SAME AS PAPER - High-Dimensional, No Correlation, Random Cov --------------------
FDR_rand_plus_MS ==> 0.07407407407407407
FDR_rand_only ==> 0.12280701754385964
FDR_MS_only ==> 0.09090909090909091
FDR_MDS_only ==> 0.10714285714285714
TPR_rand_plus_MS ==> 1.0
TPR_rand_only ==> 1.0
TPR_MS_only ==> 1.0
TPR_MDS_only ==> 1.0
POWER_rand_plus_MS ==> 1.0
POWER_rand_only ==> 1.0
POWER_MS_only ==> 1.0
POWER_MDS_only ==> 1.0


In [ ]:
Random.seed!(13)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    beta_intercept=beta_intercept,
    gamma_randomisation=1.,
    estimate_sigma2=true,
    fdr_level=0.1
);

print_metrics(scenario=scenario, results=results)

## Correlated covariates

### Low-dimensional case
30% of coefficients are 0

In [ ]:
n = 100
p = 20
prop_zero_coef=0.3
beta_intercept=1.
sigma2=1.
correlation_coefficients=[0.5, 0.3]
block_covariance=false
scenario = "Low-Dimensional, With Correlation, Random Cov"

Random.seed!(1345)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    gamma_randomisation=1.,
    fdr_level=0.1
);

In [ ]:
print_metrics(scenario=scenario, results=results)

### High-dimensional case
70% of coefficients are 0

In [ ]:
n = 100
p = 200
prop_zero_coef=0.9
beta_intercept=1.
sigma2=1.
correlation_coefficients=[0.5, 0.3]
block_covariance=false
scenario = "High-Dimensional, With Correlation, Random Cov"

Random.seed!(1345)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    gamma_randomisation=1.,
    fdr_level=0.1
);

In [ ]:
print_metrics(scenario=scenario, results=results)

#### With Block diagonal matrix
Covariates from one block belongs to S1 (coeffs != 0) and covariates from the other block belong to S0 (coeff = 0) 

In [ ]:
n = 100
p = 20
prop_zero_coef=0.5
beta_intercept=1.
sigma2=1.
correlation_coefficients=[0.5]
block_covariance = true
scenario = "Low-Dimensional, with Correlation, Block Cov"

Random.seed!(1345)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    gamma_randomisation=1.,
    fdr_level=0.1
);

In [ ]:
print_metrics(scenario=scenario, results=results)

In [ ]:
n = 100
p = 200
prop_zero_coef=0.8
beta_intercept=1.
sigma2=1.
correlation_coefficients=[0.5, 0.4]
block_covariance = true
scenario = "High-Dimensional, with Correlation, block Cov"

Random.seed!(1345)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    gamma_randomisation=1.,
    fdr_level=0.1
);

In [ ]:
print_metrics(scenario=scenario, results=results)

### High-dimensional case, 20% on non-zero coefficients, positive and negative correlations

In [9]:
n = 100
p = 200
prop_zero_coef=0.8
beta_intercept=1.
sigma2=1.
correlation_coefficients=[0.3, -0.2]
block_covariance=true
scenario = "High-Dimensional, With positive and negative Correlations and 80% of zero coefficients, Block Cov"

Random.seed!(1345)
results = wrapper_pipeline_inference.wrapper_randomisation_inference(
    n=n,
    p=p,
    correlation_coefficients=correlation_coefficients,
    block_covariance=block_covariance,
    prop_zero_coef=prop_zero_coef,
    sigma2=sigma2,
    gamma_randomisation=1.,
    fdr_level=0.1
);

In [10]:
print_metrics(scenario=scenario, results=results)

-------------------- High-Dimensional, With positive and negative Correlations and 80% of zero coefficients, Block Cov --------------------
FDR_rand_plus_MS ==> 0.11764705882352941
FDR_rand_only ==> 0.0
FDR_MS_only ==> 0.0
TPR_rand_plus_MS ==> 0.375
TPR_rand_only ==> 0.3
TPR_MS_only ==> 0.175


## Focus on High-Dimensional scenarios
Gradually increase the correlation structure and the proportion of non-zero coefficients

In [19]:
correlations_first_offdiag = [0., 0.1, 0.3, 0.5]
# correlations_second_offdiag = [0., 0.1, 0.2, 0.3, 0.4]
correlations_second_offdiag = [0.]
proportions_zero_coef = [0.95, 0.9, 0.85, 0.8]
block_covariance_options = [true, false];

In [20]:
metrics_names = String[]
for metric_name in keys(results.class_metrics)
    push!(metrics_names, String(metric_name))
end

In [21]:
metrics_names

12-element Vector{String}:
 "FDR_rand_plus_MS"
 "FDR_rand_only"
 "FDR_MS_only"
 "FDR_MDS_only"
 "TPR_rand_plus_MS"
 "TPR_rand_only"
 "TPR_MS_only"
 "TPR_MDS_only"
 "POWER_rand_plus_MS"
 "POWER_rand_only"
 "POWER_MS_only"
 "POWER_MDS_only"

In [22]:
tot_simulations = length(correlations_first_offdiag) * length(correlations_second_offdiag) * length(proportions_zero_coef) * length(block_covariance_options)
colnames = append!(["block_diagonal", "corr_first", "corr_second", "prop_non_zero"], metrics_names);
print("# simulations: $tot_simulations")

# simulations: 32

In [23]:
df_metrics = DataFrames.DataFrame([name => [] for name in colnames]);

In [24]:
df_metrics

Row,block_diagonal,corr_first,corr_second,prop_non_zero,FDR_rand_plus_MS,FDR_rand_only,FDR_MS_only,FDR_MDS_only,TPR_rand_plus_MS,TPR_rand_only,TPR_MS_only,TPR_MDS_only,POWER_rand_plus_MS,POWER_rand_only,POWER_MS_only,POWER_MDS_only
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any


In [25]:
n = 500
p = 500
beta_intercept=1.
sigma2=1.
gamma_randomisation=1.
estimate_sigma2=true
n_replica = 10

for cov_structure in block_covariance_options
    println("Executing block covariance $cov_structure")
    for corr_first in correlations_first_offdiag
        println("Executing correlation $corr_first")
        for corr_sec in correlations_second_offdiag
            for prop_zero in proportions_zero_coef

                Random.seed!(1345)
                scenario = "Correlations: $corr_first and $corr_sec. Proportion $prop_zero of zero coefficients"
                correlation_coefficients=[corr_first, corr_sec]
                
                # Initialise to 0
                average_metrics = zeros(length(keys(results.class_metrics)))

                # Do an average over n replications for each combination
                n_good_experiments = 0
                for replica in range(1, n_replica)
                    try
                        results = wrapper_pipeline_inference.wrapper_randomisation_inference(
                            n=n,
                            p=p,
                            correlation_coefficients=correlation_coefficients,
                            block_covariance=cov_structure,
                            prop_zero_coef=prop_zero,
                            sigma2=sigma2,
                            beta_intercept=beta_intercept,
                            estimate_sigma2=estimate_sigma2,
                            gamma_randomisation=1.,
                            fdr_level=0.1
                        )

                        for (metric, value) in enumerate(results.class_metrics)
                            average_metrics[metric] += value
                        end
                        n_good_experiments += 1
    
                    catch pipeline_error
                        println("Warning: ", pipeline_error)
                        continue
                    end
                    
                end

                push!(
                    df_metrics,
                    append!(
                        [
                            cov_structure,
                            corr_first,
                            corr_sec,
                            1-prop_zero
                        ],
                        average_metrics ./ n_good_experiments
                    )
                )

            end
        end
    end
end
;

Executing block covariance true
Executing correlation 0.0


Executing correlation 0.1


Executing correlation 0.3


Executing correlation 0.5


Executing block covariance false
Executing correlation 0.0


Executing correlation 0.1


Executing correlation 0.3


Executing correlation 0.5


In [26]:
df_metrics[!, "round_prop_non_zero"] = round.(df_metrics[!, "prop_non_zero"], digits=3);

In [27]:
df_metrics[1:5, :]

Row,block_diagonal,corr_first,corr_second,prop_non_zero,FDR_rand_plus_MS,FDR_rand_only,FDR_MS_only,FDR_MDS_only,TPR_rand_plus_MS,TPR_rand_only,TPR_MS_only,TPR_MDS_only,POWER_rand_plus_MS,POWER_rand_only,POWER_MS_only,POWER_MDS_only,round_prop_non_zero
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64
1,1.0,0.0,0.0,0.05,0.0867391,0.133482,0.0899972,0.0260989,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.05
2,1.0,0.0,0.0,0.1,0.0785751,0.0923231,0.101929,0.0130125,1.0,1.0,1.0,0.9,1.0,1.0,1.0,0.9,0.1
3,1.0,0.0,0.0,0.15,0.081663,0.0825535,0.102089,0.0181217,1.0,1.0,0.945333,0.994667,1.0,1.0,0.945333,0.994667,0.15
4,1.0,0.0,0.0,0.2,0.0683138,0.075368,0.178095,0.0706885,1.0,0.999,0.482,0.947,1.0,0.999,0.482,0.947,0.2
5,1.0,0.1,0.0,0.05,0.0771828,0.138054,0.0974799,0.022792,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.05


In [28]:
# CSV.write("./simulation_MS_with_randomisation_n500_p500_sigma2_estimated_new.csv", df_metrics)

# df_metrics = CSV.read("./simulation_MS_with_randomisation.csv", DataFrames.DataFrame);

"./simulation_MS_with_randomisation_n500_p500_sigma2_estimated_new.csv"